In [64]:
import requests
import pandas as pd
import json
import xarray as xr
import numpy as np

In [ ]:
f = open('leg1_data.json',)
data = json.load(f)


data['sensor']

,sensortimestamp,sensorvalue,geolocation,dataid
0,2021-08-20T11:46:00.000+00:00,4.070000,"{""Type"":""Point"",""Coordinates"":[8.7699727376302...",2021-08-20T11:46:00
1,2021-08-20T11:47:00.000+00:00,4.180000,"{""Type"":""Point"",""Coordinates"":[8.7699829101562...",2021-08-20T11:47:00
2,2021-08-20T11:48:00.000+00:00,4.100000,"{""Type"":""Point"",""Coordinates"":[8.7699788411458...",2021-08-20T11:48:00
3,2021-08-20T11:49:00.000+00:00,3.990000,"{""Type"":""Point"",""Coordinates"":[8.7699666341145...",2021-08-20T11:49:00
4,2021-08-20T11:51:00.000+00:00,3.820000,"{""Type"":""Point"",""Coordinates"":[8.7699666341145...",2021-08-20T11:51:00
...,...,...,...,...
682674,2021-09-30T06:03:00.000+00:00,-24.320000,"{""Type"":""Point"",""Coordinates"":[-15.50786113739...",2021-09-30T06:03:00
682675,2021-09-30T06:13:00.000+00:00,-24.320000,"{""Type"":""Point"",""Coordinates"":[-15.50076961517...",2021-09-30T06:13:00
682676,2021-09-30T06:16:00.000+00:00,-24.320000,"{""Type"":""Point"",""Coordinates"":[-15.49865245819...",2021-09-30T06:16:00
682677,2021-09-30T06:29:00.000+00:00,-24.309999,"{""Type"":""Point"",""Coordinates"":[-15.48853969573...",2021-09-30T06:29:00


In [65]:
nc = xr.open_dataset('/Volumes/Macaroni/dataout/smalltest5.nc')
#nc.precip.to_dataframe().to_csv('precip.csv')


In [66]:
df_sv = nc.sv.to_dataframe()


In [67]:
df_sv = df_sv.reset_index()
df_sv = df_sv.drop(columns=['channel_id', 'raw_file'])


In [68]:
#df38 = df_sv.loc[['40000.0']]
df38 = df_sv.loc[df_sv['frequency'] == 40000.0]


In [42]:
df38

,frequency,ping_time,range,latitude,longitude,sv
0,40000.0,2022-01-29 06:50:54.891,0.000000,26.755659,-29.058857,NaN
1,40000.0,2022-01-29 06:50:54.891,0.035520,26.755659,-29.058857,2.450117e+01
2,40000.0,2022-01-29 06:50:54.891,0.071040,26.755659,-29.058857,6.522895e+01
3,40000.0,2022-01-29 06:50:54.891,0.106560,26.755659,-29.058857,1.186102e+02
4,40000.0,2022-01-29 06:50:54.891,0.142080,26.755659,-29.058857,1.627317e+02
...,...,...,...,...,...,...
18737659,40000.0,2022-01-29 19:43:03.881,99.846724,24.932352,-29.073699,2.855226e-08
18737660,40000.0,2022-01-29 19:43:03.881,99.882244,24.932352,-29.073699,2.943798e-08
18737661,40000.0,2022-01-29 19:43:03.881,99.917764,24.932352,-29.073699,2.683705e-08
18737662,40000.0,2022-01-29 19:43:03.881,99.953284,24.932352,-29.073699,2.066643e-08


In [69]:
new_df = df38[['sv']].copy()

In [70]:
new_df = new_df.groupby(np.arange(len(new_df))//30).mean()


In [71]:
df3 = new_df.reset_index()

df3['index'] = df3['index'].apply(lambda x: x*30)

In [72]:
df3.set_index('index')

,sv
index,
0,1.151690e+02
30,1.512673e-04
60,2.798551e-07
90,1.228750e-07
120,2.696197e-08
...,...
18737520,7.518266e-09
18737550,1.047790e-08
18737580,1.300176e-08


In [81]:
final = pd.merge(df3, df38, left_index=True, right_index=True)

In [82]:
final = final.drop(columns=['sv_y'])

,sv_x,frequency,ping_time,range,latitude,longitude
index,,,,,,
0,1.151690e+02,40000.0,2022-01-29 06:50:54.891,0.000000,26.755659,-29.058857
30,1.512673e-04,40000.0,2022-01-29 06:50:54.891,0.035520,26.755659,-29.058857
60,2.798551e-07,40000.0,2022-01-29 06:50:54.891,0.071040,26.755659,-29.058857
90,1.228750e-07,40000.0,2022-01-29 06:50:54.891,0.106560,26.755659,-29.058857
120,2.696197e-08,40000.0,2022-01-29 06:50:54.891,0.142080,26.755659,-29.058857
...,...,...,...,...,...,...
18737520,7.518266e-09,40000.0,2022-01-29 07:16:48.881,79.848963,26.696930,-29.059244
18737550,1.047790e-08,40000.0,2022-01-29 07:16:48.881,79.884483,26.696930,-29.059244
18737580,1.300176e-08,40000.0,2022-01-29 07:16:48.881,79.920003,26.696930,-29.059244
